In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import sys
import pickle
import tensorflow as tf
from sklearn.utils import shuffle
import tsc_utils as tu
import time
import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
g = tf.Graph()
with g.as_default():
    model = tu.lenet
    model_params = tu.params_orig_lenet
    
    model = tu.lenet
    model_params = tu.params_big_lenet

    model = tu.lenet
    model_params = tu.params_huge_lenet
    
    model = tu.sermanet
    model_params = tu.params_sermanet
    
    model = tu.sermanet
    model_params = tu.params_sermanet_big
    
    model = tu.sermanet_v2
    model_params = tu.params_sermanet_v2
    
    model = tu.sermanet_v2
    model_params = tu.params_sermanet_v2_big
    
    x = tf.placeholder(tf.float32, (None, 32, 32, 1))
    y = tf.placeholder(tf.int32, (None))
    one_hot_y = tf.one_hot(y, model_params['num_classes'])
    is_training = tf.placeholder(tf.bool)

    logits = model(x, params=model_params, is_training=is_training)

{'conv3_k': 5, 'model_name': 'sermanet_v2', 'conv3_p': 0.7, 'conv2_d': 128, 'conv1_d': 64, 'conv3_d': 256, 'fc4_size': 2048, 'conv1_k': 5, 'conv2_k': 5, 'conv2_p': 0.8, 'num_classes': 43, 'fc4_p': 0.5, 'conv1_p': 0.9}


In [24]:
with tf.Session(graph=g) as sess:
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print(variable)
        print(shape)
        #print(len(shape))
        variable_parametes = 1
        for dim in shape:
            #print(dim)
            variable_parametes *= dim.value
        #print(variable_parametes)
        total_parameters += variable_parametes
    print(total_parameters)

<tf.Variable 'conv1/weights:0' shape=(5, 5, 1, 64) dtype=float32_ref>
(5, 5, 1, 64)
<tf.Variable 'conv1/biases:0' shape=(64,) dtype=float32_ref>
(64,)
<tf.Variable 'conv2/weights:0' shape=(5, 5, 64, 128) dtype=float32_ref>
(5, 5, 64, 128)
<tf.Variable 'conv2/biases:0' shape=(128,) dtype=float32_ref>
(128,)
<tf.Variable 'conv3/weights:0' shape=(5, 5, 128, 256) dtype=float32_ref>
(5, 5, 128, 256)
<tf.Variable 'conv3/biases:0' shape=(256,) dtype=float32_ref>
(256,)
<tf.Variable 'fc4/weights:0' shape=(7168, 2048) dtype=float32_ref>
(7168, 2048)
<tf.Variable 'fc4/biases:0' shape=(2048,) dtype=float32_ref>
(2048,)
<tf.Variable 'out/weights:0' shape=(2048, 43) dtype=float32_ref>
(2048, 43)
<tf.Variable 'out/biases:0' shape=(43,) dtype=float32_ref>
(43,)
15796267


In [4]:
# load data

# original set
train_fname = 'data/train_preprocessed.p'
valid_fname = 'data/valid_preprocessed.p'

# augmented set with smaller variations
train_fname_FA_fine = 'data/train_flip_extended_augmented_0.7_preprocessed.p'
valid_fname_FA_fine = 'data/valid_flip_extended_augmented_0.7_preprocessed.p'

# augmented set
train_fname_FA = 'data/train_flip_extended_augmented_preprocessed.p'
valid_fname_FA = 'data/valid_flip_extended_augmented_preprocessed.p'

# flip extended set
train_fname_F = 'data/train_flip_extended_preprocessed.p'
valid_fname_F = 'data/valid_flip_extended_preprocessed.p'


test_fname = 'data/test_preprocessed.p'

def data_loader(train_file, valid_file, test_file):
    X_train, y_train = tu.load_pickled_data(train_file, columns = ['features', 'labels'])
    X_valid, y_valid = tu.load_pickled_data(valid_file, columns = ['features', 'labels'])
    X_test, y_test = tu.load_pickled_data(test_file, columns = ['features', 'labels'])
    print(X_train.shape, y_train.shape)
    print(X_valid.shape, y_valid.shape)
    print(X_test.shape, y_test.shape)
    return (X_train, y_train, X_valid, y_valid, X_test, y_test)

In [5]:
data_set0 = data_loader(train_fname, valid_fname, test_fname)

(34799, 32, 32, 1) (34799,)
(4410, 32, 32, 1) (4410,)
(12630, 32, 32, 1) (12630,)


In [6]:
model = tu.lenet
params = tu.params_orig_lenet
params['batch_norm'] = True
params['conv1_p'] = 1
params['conv2_p'] = 1
params['fc3_p'] = 1
params['fc4_p'] = 1
config = dict(model=model, model_params=params, resuming=False)

_ = tu.train_model(*data_set0, **config, learning_rate=0.001)

========= train_model() arguments: ==========
resuming = False
model = <function lenet at 0x00000247D1670E18>
model_params = {'batch_norm': True, 'conv2_p': 1, 'model_name': 'lenet', 'fc3_p': 1, 'conv1_d': 6, 'num_classes': 43, 'conv1_k': 5, 'conv2_k': 5, 'conv2_d': 16, 'fc3_size': 120, 'fc4_size': 84, 'fc4_p': 1, 'conv1_p': 1}
learning_rate = 0.001
max_epochs = 1001
batch_size = 256
early_stopping_enabled = True
early_stopping_patience = 10
log_epoch = 1
print_epoch = 1
top_k = 5
return_top_k = False
plot_featuremap = False
BN_lenet__conv1_d_6_conv1_k_5_conv1_p_1_conv2_d_16_conv2_k_5_conv2_p_1_fc3_p_1_fc3_size_120_fc4_p_1_fc4_size_84
a8045a95509b7c74
model dir: C:\Users\dingran\github\CarND-Traffic-Sign-Classifier-Project\models\a8045a95509b7c74
{'batch_norm': True, 'conv2_p': 1, 'model_name': 'lenet', 'fc3_p': 1, 'conv1_d': 6, 'num_classes': 43, 'conv1_k': 5, 'conv2_k': 5, 'conv2_d': 16, 'fc3_size': 120, 'fc4_size': 84, 'fc4_p': 1, 'conv1_p': 1}
lenet pool2 reshaped size:  [None, 102

KeyboardInterrupt: 